In [49]:
import osmnx as ox
import geopandas as gpd
import pandas as pd

In [50]:
G = ox.load_graphml("bangkok.graphml")

In [52]:

tags = {
    "amenity": [
        "hospital",
        "fire_station",
        "police",
        "bus_station",
        "college",
        "university",
    ],
    "shop": "mall",
    # "leisure": "stadium",
    # "railway": "station",
}

# Download everything in one pass
print("downloading POIs")
pois = ox.features_from_place("Bangkok, Thailand", tags=tags)

# Compute lat/lon
print("computing lat/lon")
pois["lat"] = pois.geometry.centroid.y
pois["lon"] = pois.geometry.centroid.x

# Keep only columns we care about
pois = pois[["name", "lat", "lon", "amenity", "shop", "leisure", "railway"]]


downloading POIs
computing lat/lon


/opt/anaconda3/envs/311_final/lib/python3.10/site-packages/shapely/predicates.py:878: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/opt/anaconda3/envs/311_final/lib/python3.10/site-packages/shapely/set_operations.py:451: RuntimeWarning: invalid value encountered in union
  return lib.union(a, b, **kwargs)
/opt/anaconda3/envs/311_final/lib/python3.10/site-packages/shapely/predicates.py:878: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/opt/anaconda3/envs/311_final/lib/python3.10/site-packages/shapely/predicates.py:878: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/opt/anaconda3/envs/311_final/lib/python3.10/site-packages/shapely/predicates.py:878: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/opt/anaconda3/envs/311_final/lib/python3.10/site-packages/shapely/predicates.py:878: RuntimeWar

In [10]:
HIGH_VALUE = [
    "hospital",
    "fire_station",
    "police",
    "mall",
    "bus_station",
    "university",
    "college",
    "station",   # railway / MRT / BTS stations
]

pois_hq = pois[pois["category"].isin(HIGH_VALUE)].copy()
print(len(pois_hq))


766


/opt/anaconda3/envs/311_final/lib/python3.10/site-packages/shapely/constructive.py:246: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(
/opt/anaconda3/envs/311_final/lib/python3.10/site-packages/shapely/predicates.py:878: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/opt/anaconda3/envs/311_final/lib/python3.10/site-packages/shapely/set_operations.py:451: RuntimeWarning: invalid value encountered in union
  return lib.union(a, b, **kwargs)
/opt/anaconda3/envs/311_final/lib/python3.10/site-packages/shapely/predicates.py:878: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/opt/anaconda3/envs/311_final/lib/python3.10/site-packages/shapely/set_operations.py:451: RuntimeWarning: invalid value encountered in union
  return lib.union(a, b, **kwargs)


In [17]:
print(G)

MultiDiGraph with 5889 nodes and 9636 edges


In [25]:
print(len(G.nodes), len(G.edges))
print(any("geometry" in d for _,_,d in G.edges(data=True)))

5889 9636
True


In [40]:
G_simplified = ox.consolidate_intersections(
    G,
    tolerance=0.001,      # how close nodes must be to merge (10–20m typical)
    rebuild_graph=True # << this triggers graph reconstruction
)

print(G_simplified)

/opt/anaconda3/envs/311_final/lib/python3.10/site-packages/osmnx/simplification.py:596: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  merged = gdf_nodes.buffer(tolerance).union_all()
/opt/anaconda3/envs/311_final/lib/python3.10/site-packages/shapely/set_operations.py:553: RuntimeWarning: invalid value encountered in unary_union
  return lib.unary_union(collections, **kwargs)
/opt/anaconda3/envs/311_final/lib/python3.10/site-packages/osmnx/simplification.py:655: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids = node_clusters.centroid


MultiDiGraph with 1392 nodes and 3098 edges


In [48]:
nodes_gdf, edges_gdf = ox.graph_to_gdfs(G_simplified)
street_counts = ox.stats.streets_per_node(G_simplified)
# add street_count column
nodes_gdf['street_count'] = nodes_gdf.index.map(street_counts)

# filter true intersections
real_ints = nodes_gdf[nodes_gdf['street_count'] >= 3]

print("True intersections:", len(real_ints))


True intersections: 1059
